# Día 2: Métodos Mágicos (Magic Methods)

## Descripción General

Los métodos mágicos (también conocidos como *dunder methods* por "double underscore") son métodos especiales en Python que comienzan y terminan con doble guion bajo (`__`). Estos métodos permiten que tus clases personalizadas se integren perfectamente con las operaciones integradas de Python, como la impresión, comparación, iteración y operaciones aritméticas.

En este notebook, exploraremos los métodos mágicos más comunes y útiles que te permitirán crear clases que se comportan de manera natural y pythónica.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Comprender qué son los métodos mágicos y por qué son importantes en Python
2. Implementar `__str__` y `__repr__` para controlar la representación de objetos
3. Usar `__eq__`, `__lt__`, y otros métodos de comparación para definir igualdad y orden
4. Implementar `__len__` y `__getitem__` para crear objetos tipo contenedor
5. Utilizar `__iter__` y `__next__` para hacer objetos iterables

## 1. Introducción a los Métodos Mágicos

### El Problema que Resuelve

Sin métodos mágicos, nuestras clases personalizadas no se integran bien con las operaciones integradas de Python. Por ejemplo, no podemos imprimirlas de manera legible, compararlas, o usarlas en loops.

In [ ]:
# BAD: Class without magic methods
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

p1 = Point(3, 4)
print(p1)  # Output: <__main__.Point object at 0x...> - Not helpful!
print(p1 == Point(3, 4))  # False - Different objects, even with same values

### Aprendizaje Clave

Los métodos mágicos son la interfaz entre tus clases y el modelo de datos de Python. Permiten que tus objetos se comporten como tipos integrados, haciendo tu código más intuitivo y pythónico.

**Referencia oficial:** [Data Model - Python Documentation](https://docs.python.org/3/reference/datamodel.html)

## 2. __str__ y __repr__: Representación de Objetos

### El Problema que Resuelve

Cuando imprimes un objeto o lo inspeccionas en el REPL, Python necesita saber cómo convertirlo a string. `__str__` proporciona una representación "amigable" para usuarios, mientras que `__repr__` proporciona una representación "técnica" para desarrolladores.

In [ ]:
# GOOD: Class with __str__ and __repr__
class Point:
    """
    Represent a point in 2D space.
    
    :param x: X coordinate
    :type x: float
    :param y: Y coordinate
    :type y: float
    """
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y
    
    def __str__(self) -> str:
        """Return user-friendly string representation."""
        return f"Point at ({self.x}, {self.y})"
    
    def __repr__(self) -> str:
        """Return developer-friendly string representation."""
        return f"Point({self.x}, {self.y})"

p1 = Point(3, 4)
print(p1)  # Uses __str__: Point at (3, 4)
print(repr(p1))  # Uses __repr__: Point(3, 4)
print([p1])  # Lists use __repr__: [Point(3, 4)]

### Aprendizaje Clave

`__str__` es para usuarios finales (legible), `__repr__` es para desarrolladores (sin ambigüedad, idealmente evaluable). Si solo implementas uno, implementa `__repr__` porque Python lo usará como fallback para `__str__`.

**Referencia oficial:** [object.__str__ and object.__repr__](https://docs.python.org/3/reference/datamodel.html#object.__repr__)

### Pregunta de Comprensión

¿Cuál es la diferencia entre `__str__` y `__repr__`, y cuándo se usa cada uno?

## 3. Métodos de Comparación: __eq__, __lt__, __le__, __gt__, __ge__

### El Problema que Resuelve

Por defecto, Python compara objetos por identidad (si son el mismo objeto en memoria). Los métodos de comparación permiten definir igualdad y orden basados en los valores de los atributos.

In [ ]:
# BAD: Without comparison methods
class Person:
    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

p1 = Person("Alice", 30)
p2 = Person("Alice", 30)
print(p1 == p2)  # False - Different objects!
# print(p1 < p2)  # TypeError: '<' not supported

In [ ]:
# GOOD: With comparison methods
class Person:
    """
    Represent a person with name and age.
    
    :param name: Person's name
    :type name: str
    :param age: Person's age
    :type age: int
    """
    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age
    
    def __eq__(self, other: object) -> bool:
        """Check equality based on name and age."""
        if not isinstance(other, Person):
            return NotImplemented
        return self.name == other.name and self.age == other.age
    
    def __lt__(self, other: 'Person') -> bool:
        """Compare by age for sorting."""
        if not isinstance(other, Person):
            return NotImplemented
        return self.age < other.age
    
    def __repr__(self) -> str:
        return f"Person('{self.name}', {self.age})"

p1 = Person("Alice", 30)
p2 = Person("Alice", 30)
p3 = Person("Bob", 25)

print(p1 == p2)  # True - Same values!
print(p1 < p3)  # False - Alice is older
print(p3 < p1)  # True - Bob is younger

# Sorting works now!
people = [p1, p3, Person("Charlie", 35)]
print(sorted(people))  # Sorted by age

### Aprendizaje Clave

Implementa `__eq__` para igualdad y `__lt__` para orden. Python puede derivar automáticamente `__le__`, `__gt__`, `__ge__` si usas el decorador `@functools.total_ordering`. Siempre retorna `NotImplemented` para tipos incompatibles.

**Referencia oficial:** [Rich Comparison Methods](https://docs.python.org/3/reference/datamodel.html#object.__lt__)

In [ ]:
# Using @total_ordering to reduce boilerplate
from functools import total_ordering

@total_ordering
class Product:
    """
    Represent a product with name and price.
    
    :param name: Product name
    :type name: str
    :param price: Product price
    :type price: float
    """
    def __init__(self, name: str, price: float) -> None:
        self.name = name
        self.price = price
    
    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Product):
            return NotImplemented
        return self.price == other.price
    
    def __lt__(self, other: 'Product') -> bool:
        if not isinstance(other, Product):
            return NotImplemented
        return self.price < other.price
    
    def __repr__(self) -> str:
        return f"Product('{self.name}', ${self.price})"

# Now all comparison operators work!
apple = Product("Apple", 0.5)
banana = Product("Banana", 0.3)
orange = Product("Orange", 0.8)

print(apple > banana)  # True
print(apple <= orange)  # True
print(banana >= orange)  # False

## 4. Métodos de Contenedor: __len__ y __getitem__

### El Problema que Resuelve

Para crear clases que se comportan como contenedores (listas, diccionarios), necesitamos implementar métodos que permitan acceder a elementos, obtener la longitud, y usar indexación.

In [ ]:
# GOOD: Custom container class
class Playlist:
    """
    Represent a music playlist.
    
    :param name: Playlist name
    :type name: str
    """
    def __init__(self, name: str) -> None:
        self.name = name
        self._songs = []
    
    def add_song(self, song: str) -> None:
        """Add a song to the playlist."""
        self._songs.append(song)
    
    def __len__(self) -> int:
        """Return number of songs in playlist."""
        return len(self._songs)
    
    def __getitem__(self, index: int) -> str:
        """Get song at index."""
        return self._songs[index]
    
    def __repr__(self) -> str:
        return f"Playlist('{self.name}', {len(self)} songs)"

playlist = Playlist("My Favorites")
playlist.add_song("Song A")
playlist.add_song("Song B")
playlist.add_song("Song C")

print(len(playlist))  # 3 - __len__ works!
print(playlist[0])  # "Song A" - __getitem__ works!
print(playlist[-1])  # "Song C" - Negative indexing works!

# Can use in for loop because __getitem__ is defined
for song in playlist:
    print(f"Playing: {song}")

### Aprendizaje Clave

`__len__` permite usar `len()` en tu objeto. `__getitem__` permite indexación con `[]` y hace tu objeto iterable automáticamente. También puedes implementar `__setitem__` y `__delitem__` para modificación y eliminación.

**Referencia oficial:** [Emulating Container Types](https://docs.python.org/3/reference/datamodel.html#emulating-container-types)

### Pregunta de Comprensión

¿Qué métodos mágicos necesitas implementar para que tu clase se comporte como una lista?

## 5. Métodos de Iteración: __iter__ y __next__

### El Problema que Resuelve

Para crear objetos que pueden ser iterados en loops `for`, necesitamos implementar el protocolo de iteración. Esto es más flexible que solo implementar `__getitem__`.

In [ ]:
# GOOD: Custom iterator
class Countdown:
    """
    Iterator that counts down from a starting number.
    
    :param start: Starting number
    :type start: int
    """
    def __init__(self, start: int) -> None:
        self.start = start
        self.current = start
    
    def __iter__(self) -> 'Countdown':
        """Return the iterator object (self)."""
        self.current = self.start  # Reset for reuse
        return self
    
    def __next__(self) -> int:
        """Return the next value in the countdown."""
        if self.current < 0:
            raise StopIteration
        value = self.current
        self.current -= 1
        return value

countdown = Countdown(5)
for num in countdown:
    print(num)  # 5, 4, 3, 2, 1, 0

# Can iterate again
print(list(countdown))  # [5, 4, 3, 2, 1, 0]

### Aprendizaje Clave

`__iter__` debe retornar un objeto iterador (usualmente `self`). `__next__` retorna el siguiente valor o lanza `StopIteration` cuando no hay más elementos. Este patrón permite iteración personalizada y lazy evaluation.

**Referencia oficial:** [Iterator Types](https://docs.python.org/3/library/stdtypes.html#iterator-types)

In [ ]:
# Example: Range-like class
class CustomRange:
    """
    Custom implementation of range().
    
    :param start: Starting value
    :type start: int
    :param stop: Stopping value (exclusive)
    :type stop: int
    :param step: Step size
    :type step: int
    """
    def __init__(self, start: int, stop: int, step: int = 1) -> None:
        self.start = start
        self.stop = stop
        self.step = step
    
    def __iter__(self) -> 'CustomRange':
        self.current = self.start
        return self
    
    def __next__(self) -> int:
        if self.current >= self.stop:
            raise StopIteration
        value = self.current
        self.current += self.step
        return value
    
    def __repr__(self) -> str:
        return f"CustomRange({self.start}, {self.stop}, {self.step})"

# Works like range()
for i in CustomRange(0, 10, 2):
    print(i)  # 0, 2, 4, 6, 8

## 6. Otros Métodos Mágicos Útiles

### __bool__: Valor de Verdad

Define cómo tu objeto se evalúa en contextos booleanos (if, while, etc.).

In [ ]:
class ShoppingCart:
    """
    Represent a shopping cart.
    """
    def __init__(self) -> None:
        self.items = []
    
    def add_item(self, item: str) -> None:
        """Add item to cart."""
        self.items.append(item)
    
    def __bool__(self) -> bool:
        """Cart is truthy if it has items."""
        return len(self.items) > 0
    
    def __len__(self) -> int:
        return len(self.items)

cart = ShoppingCart()
if cart:
    print("Cart has items")
else:
    print("Cart is empty")  # This prints

cart.add_item("Apple")
if cart:
    print("Cart has items")  # This prints now

### __call__: Objetos Llamables

Permite que instancias de tu clase sean llamadas como funciones.

In [ ]:
class Multiplier:
    """
    Callable object that multiplies by a factor.
    
    :param factor: Multiplication factor
    :type factor: float
    """
    def __init__(self, factor: float) -> None:
        self.factor = factor
    
    def __call__(self, value: float) -> float:
        """Multiply value by factor."""
        return value * self.factor

double = Multiplier(2)
triple = Multiplier(3)

print(double(5))  # 10 - Called like a function!
print(triple(5))  # 15

# Useful for creating function-like objects with state
numbers = [1, 2, 3, 4, 5]
print(list(map(double, numbers)))  # [2, 4, 6, 8, 10]

### __contains__: Operador 'in'

Define el comportamiento del operador `in` para tu clase.

In [ ]:
class Team:
    """
    Represent a team of members.
    
    :param name: Team name
    :type name: str
    """
    def __init__(self, name: str) -> None:
        self.name = name
        self.members = set()
    
    def add_member(self, member: str) -> None:
        """Add member to team."""
        self.members.add(member)
    
    def __contains__(self, member: str) -> bool:
        """Check if member is in team."""
        return member in self.members

team = Team("Python Developers")
team.add_member("Alice")
team.add_member("Bob")

print("Alice" in team)  # True
print("Charlie" in team)  # False

### Aprendizaje Clave

Hay muchos más métodos mágicos: `__add__` para `+`, `__hash__` para usar objetos en sets/dicts, `__enter__`/`__exit__` para context managers. Implementa solo los que necesites para hacer tu clase intuitiva.

**Referencia oficial:** [Special Method Names](https://docs.python.org/3/reference/datamodel.html#special-method-names)

### Respuesta a la Pregunta Anterior

**¿Qué métodos para comportarse como lista?** Mínimo: `__len__` y `__getitem__`. Para funcionalidad completa: también `__setitem__`, `__delitem__`, `__iter__`, `__contains__`, `append()`, `insert()`, etc. Considera heredar de `collections.abc.MutableSequence` para obtener implementaciones por defecto.

## Ejercicios Prácticos

### Tarea 1: Representación de Objetos

Completa los ejercicios en `day_2/exercises/06_magic_methods.py`:

1. Implementa `__str__` y `__repr__` para una clase `Book`
2. Asegúrate de que `__repr__` sea evaluable (pueda recrear el objeto)

### Tarea 2: Comparación de Objetos

1. Implementa `__eq__` y `__lt__` para una clase `Student`
2. Usa `@total_ordering` para obtener todos los operadores de comparación
3. Haz que los estudiantes se ordenen por calificación

### Tarea 3: Contenedores Personalizados

1. Crea una clase `Library` que se comporte como un contenedor
2. Implementa `__len__`, `__getitem__`, y `__contains__`
3. Permite iterar sobre los libros en la biblioteca

### Tarea 4: Iteradores Personalizados

1. Crea un iterador `EvenNumbers` que genere números pares
2. Implementa `__iter__` y `__next__`
3. Asegúrate de que lance `StopIteration` apropiadamente

Ejecuta los tests con:
```bash
pytest day_2/exercises/tests/test_06_magic_methods.py
```

## Resumen

En este notebook has aprendido:

1. **Métodos mágicos** permiten que tus clases se integren con las operaciones integradas de Python
2. **`__str__` y `__repr__`** controlan cómo se representan tus objetos como strings
3. **Métodos de comparación** (`__eq__`, `__lt__`, etc.) permiten comparar y ordenar objetos
4. **`__len__` y `__getitem__`** hacen que tus objetos se comporten como contenedores
5. **`__iter__` y `__next__`** implementan el protocolo de iteración para loops personalizados

### Próximos Pasos

Los métodos mágicos son fundamentales para crear clases pythónicas. En los siguientes notebooks, veremos cómo combinar estos conceptos con otros patrones de diseño para crear código más elegante y mantenible.

## Preguntas de Autoevaluación

### 1. ¿Cuál es la diferencia entre `__str__` y `__repr__`?

**Respuesta:** `__str__` proporciona una representación "amigable" para usuarios finales (legible), mientras que `__repr__` proporciona una representación "técnica" para desarrolladores (sin ambigüedad, idealmente evaluable). `print()` usa `__str__`, mientras que el REPL y `repr()` usan `__repr__`.

### 2. ¿Qué debe retornar un método de comparación cuando se compara con un tipo incompatible?

**Respuesta:** Debe retornar `NotImplemented` (no `False` o lanzar excepción). Esto permite que Python intente la comparación inversa o use el método del otro objeto.

### 3. ¿Qué hace el decorador `@functools.total_ordering`?

**Respuesta:** Genera automáticamente los métodos de comparación faltantes (`__le__`, `__gt__`, `__ge__`) si defines `__eq__` y uno de `__lt__`, `__le__`, `__gt__`, o `__ge__`. Reduce el boilerplate code.

### 4. ¿Cuál es la diferencia entre implementar `__getitem__` y `__iter__`?

**Respuesta:** `__getitem__` permite indexación con `[]` y hace el objeto iterable automáticamente (Python itera usando índices 0, 1, 2...). `__iter__` proporciona control más fino sobre la iteración y es más eficiente para objetos que no son secuencias.

### 5. ¿Cuándo debe un método `__next__` lanzar `StopIteration`?

**Respuesta:** Cuando no hay más elementos para iterar. Esta excepción señala a Python que la iteración ha terminado y es parte del protocolo de iteración.

### 6. ¿Qué método mágico permite usar el operador `in` con tu clase?

**Respuesta:** `__contains__`. Debe retornar `True` si el elemento está en el contenedor, `False` en caso contrario. Si no está implementado, Python intentará iterar sobre el objeto usando `__iter__` o `__getitem__`.

### 7. ¿Para qué sirve el método `__call__`?

**Respuesta:** Permite que instancias de tu clase sean llamadas como funciones. Es útil para crear objetos con estado que se comportan como funciones, como closures o functors.

Discute tus respuestas con compañeros para profundizar tu comprensión.

## Recursos y Referencias Oficiales

### Documentación Oficial

- **Data Model - Python Documentation**: [https://docs.python.org/3/reference/datamodel.html](https://docs.python.org/3/reference/datamodel.html)
  - Documentación completa del modelo de datos de Python
  - Incluye todos los métodos mágicos disponibles

- **Special Method Names**: [https://docs.python.org/3/reference/datamodel.html#special-method-names](https://docs.python.org/3/reference/datamodel.html#special-method-names)
  - Lista completa de métodos especiales
  - Explicación detallada de cada uno

- **Iterator Types**: [https://docs.python.org/3/library/stdtypes.html#iterator-types](https://docs.python.org/3/library/stdtypes.html#iterator-types)
  - Documentación sobre el protocolo de iteración

- **Emulating Container Types**: [https://docs.python.org/3/reference/datamodel.html#emulating-container-types](https://docs.python.org/3/reference/datamodel.html#emulating-container-types)
  - Guía para crear contenedores personalizados

### Módulos Relacionados

- **functools.total_ordering**: [https://docs.python.org/3/library/functools.html#functools.total_ordering](https://docs.python.org/3/library/functools.html#functools.total_ordering)
  - Decorador para generar métodos de comparación

- **collections.abc**: [https://docs.python.org/3/library/collections.abc.html](https://docs.python.org/3/library/collections.abc.html)
  - Abstract Base Classes para contenedores
  - Proporciona implementaciones por defecto de métodos

### Mejores Prácticas

- **Real Python - Python Magic Methods**: [https://realpython.com/python-magic-methods/](https://realpython.com/python-magic-methods/)
  - Tutorial completo con ejemplos prácticos

- **Python Morsels - Every Dunder Method**: [https://www.pythonmorsels.com/every-dunder-method/](https://www.pythonmorsels.com/every-dunder-method/)
  - Lista exhaustiva de todos los métodos mágicos

- **Python Guide - Object-Oriented Programming**: [https://docs.python-guide.org/writing/structure/#object-oriented-programming](https://docs.python-guide.org/writing/structure/#object-oriented-programming)
  - Guía de estilo y mejores prácticas

### Artículos Avanzados

- **Descriptor HowTo Guide**: [https://docs.python.org/3/howto/descriptor.html](https://docs.python.org/3/howto/descriptor.html)
  - Para métodos mágicos avanzados como `__get__`, `__set__`, `__delete__`

- **Python's Data Model**: [https://docs.python.org/3/reference/datamodel.html](https://docs.python.org/3/reference/datamodel.html)
  - Referencia completa del modelo de datos

### Notas Importantes

- Todos los enlaces están actualizados a partir de 2024
- Se recomienda revisar la documentación oficial regularmente
- El Data Model es la fuente más confiable para entender métodos mágicos